In [2]:
import pandas as pd
import numpy as np
import glob
import gc
import keras 
from keras.layers import LSTM, Dense, TimeDistributed, Dropout, MaxPooling2D, Conv2D, Input
from keras.layers import Flatten, BatchNormalization, Activation, Reshape, concatenate
from keras.models import Model
import random
import keras.backend as K

In [3]:
height = 108
width = 108
steps = 20

In [4]:
def classifier_5(optimizer='adam', loss='binary_crossentropy', metrix='accuracy'):
    #cnn
    cnn_inputs = Input(shape=(height, width, 1), name='cnn_inputs')
    layers = BatchNormalization()(cnn_inputs)
    layers = Conv2D(32, (3,3), activation="relu")(layers)
    layers = Conv2D(32, (3,3), activation="relu")(layers)
    layers = MaxPooling2D(2,2)(layers)
    layers = Conv2D(64, (3,3), activation='relu')(layers)
    layers = Conv2D(64, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2)(layers)
    layers = Conv2D(128, (3,3), activation='relu')(layers)
    layers = Conv2D(128, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = Conv2D(256, (1,1), activation='relu')(layers)
    layers = Flatten()(layers)
    layers = Dropout(0.3)(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #lstm
    lstm_inputs = Input(shape=(steps, 24), name='lstm_inputs')
    lstm_layers = LSTM(16, return_sequences=True)(lstm_inputs)
    lstm_layers = LSTM(16)(lstm_layers)
    lstm_layers = Dense(24, activation='relu')(lstm_layers)
    
    #main route
    main_ = concatenate([lstm_layers, layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[lstm_inputs, cnn_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss=loss, metrics=[metrix])
    return model

In [5]:
classifier = classifier_5(optimizer='adam')
classifier.load_weights('weights/weight-24units-model4-enopadding.hdf5')

/anaconda3/envs/nd2/lib/python2.7/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
/anaconda3/envs/nd2/lib/python2.7/site-packages/ipykernel/__main__.py:9: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`


In [6]:
def give_prediction(classifier, cnn_input, lstm_input):

    prediction = classifier.predict([cnn_input, lstm_input])
    output = 'The notes are: C4'
    for i in range(prediction.shape[0]):
        for j in range(prediction.shape[1]):
            if prediction[i][j] < 0.5:
                prediction[i][j] = 0
            elif prediction[i][j] > 0.5:
                prediction[i][j] = 1
                output+='_{}'.format(j+60)
#         print(output)
    return prediction

In [8]:
print(give_prediction(classifier, np.zeros((1,20,24)), np.zeros((1,108,108,1))))

The notes are: C4
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
